In [3]:
import pandas as pd
import sqlite3


In [4]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [5]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
merged_df = orders.merge(
    users,
    on="user_id",
    how="left"
)


In [8]:
final_df = merged_df.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [9]:
final_df[final_df['membership'] == 'Gold'].shape[0]


4987

In [10]:
final_df['total_amount'].mean()


np.float64(801.162412)

In [11]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)
print("File saved successfully!")


File saved successfully!


In [12]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

# Convert date column (important for quarter analysis)
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [13]:
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
)

gold_city_revenue


city
Bangalore     994702.59
Chennai      1080909.79
Hyderabad     896740.19
Pune         1003012.32
Name: total_amount, dtype: float64

In [14]:
gold_city_revenue.idxmax()



'Chennai'

In [15]:
cuisine_avg = df.groupby("cuisine")["total_amount"].mean()

cuisine_avg
cuisine_avg.idxmax()


'Mexican'

In [16]:
user_total = df.groupby("user_id")["total_amount"].sum()

high_value_users = user_total[user_total > 1000]

len(high_value_users)


2544

In [18]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

rating_revenue = df.groupby("rating_range")["total_amount"].sum()

rating_revenue
rating_revenue.idxmax()


'4.6 – 5.0'

In [19]:
gold_city_avg = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
)

gold_city_avg
gold_city_avg.idxmax()


'Chennai'

In [20]:
cuisine_stats = df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id", "nunique"),
    revenue=("total_amount", "sum")
)

cuisine_stats
cuisine_stats.sort_values(
    ["restaurant_count", "revenue"],
    ascending=[True, False]
).head(1)


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65


In [21]:
gold_percentage = round(
    (df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100
)

gold_percentage


50

In [22]:
restaurant_stats = df.groupby("restaurant_name_y").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

filtered_restaurants = restaurant_stats[restaurant_stats["total_orders"] < 20]

filtered_restaurants.sort_values(
    "avg_order_value",
    ascending=False
).head(1)


,avg_order_value,total_orders
restaurant_name_y,,
Restaurant_294,1040.222308,13


In [23]:
combo_revenue = (
    df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
)

combo_revenue
combo_revenue.idxmax()



('Regular', 'Mexican')

In [25]:
df["quarter"] = df["order_date"].dt.to_period("Q")

quarter_revenue = df.groupby("quarter")["total_amount"].sum()

quarter_revenue
quarter_revenue.idxmax()


Period('2023Q3', 'Q-DEC')

In [26]:
gold_orders = df[df["membership"] == "Gold"].shape[0]
gold_orders


4987

In [27]:
hyderabad_revenue = round(
    df[df["city"] == "Hyderabad"]["total_amount"].sum()
)

hyderabad_revenue


1889367

In [28]:
distinct_users = df["user_id"].nunique()
distinct_users


2883

In [29]:
gold_avg_order = round(
    df[df["membership"] == "Gold"]["total_amount"].mean(), 2
)

gold_avg_order


np.float64(797.15)

In [30]:
orders_high_rating = df[df["rating"] >= 4.5].shape[0]
orders_high_rating


3374

In [31]:

top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city


'Chennai'

In [32]:
# Orders in that city (Gold only)
orders_top_gold_city = df[
    (df["membership"] == "Gold") & (df["city"] == top_gold_city)
].shape[0]

orders_top_gold_city


1337